In [ ]:
import pandas as pd

file_path = 'Section1.xlsx'

df = pd.read_excel(file_path)

print(df.shape)
print(df.columns)
df.head(10)


(4598, 10)
Index(['Customer_ID', 'Company_Size', 'Industry', 'Past_Purchases',
       'Product_Type', 'Max_Concurrent_Users', 'Demo_Type',
       'Free_Support_Offered', 'Price_Offered', 'Purchase'],
      dtype='object')


,Customer_ID,Company_Size,Industry,Past_Purchases,Product_Type,Max_Concurrent_Users,Demo_Type,Free_Support_Offered,Price_Offered,Purchase
0,1001,103,Tech,1,Standard,75,Lite,0,1390,0
1,1002,436,Retail,3,Standard,428,Full,0,6999,0
2,1003,349,Tech,1,Premium,324,Full,1,5437,0
3,1004,271,Retail,2,Premium,155,Lite,0,2805,0
4,1005,107,Tech,1,Standard,64,Lite,0,1316,0
5,1006,72,Tech,0,Premium,96,Lite,0,1776,1
6,1007,189,Finance,0,Premium,139,Lite,0,2516,1
7,1008,21,Retail,3,Premium,14,Full,1,460,1
8,1009,103,Tech,2,Premium,103,Full,1,1862,1
9,1010,122,Retail,0,Premium,154,Lite,0,2761,0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# خواندن داده‌ی آموزش
train_df = pd.read_excel('Section1.xlsx')

# پیش‌پردازش: تبدیل categorical‌ها به one-hot
categorical_cols = ['Industry', 'Product_Type', 'Demo_Type']
train_df = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)

# تعریف X و y
X_train = train_df.drop(columns=['Customer_ID', 'Purchase'])
y_train = train_df['Purchase']

# آموزش مدل
model = RandomForestClassifier()
model.fit(X_train, y_train)

# تابع هزینه پشتیبانی
def support_cost(company_size):
    if company_size <= 20:
        return 150
    elif company_size <= 50:
        return 250
    elif company_size <= 200:
        return 400
    else:
        return 450

# ساخت لیست ستون‌های one-hot مورد نیاز
needed_columns = X_train.columns.tolist()

# تابع پیدا کردن بهترین پیشنهاد برای یک مشتری
def best_offer(customer_row):
    base = customer_row.copy()
    base.pop('Customer_ID', None)  # حذف آی‌دی از ورودی

    best_profit = -np.inf
    best_offer_result = {'make_offer': False}

    for demo_type in ['Full', 'Lite']:
        for support in [0, 1] if demo_type == 'Full' else [0]:
            for price in range(1000, 10001, 500):
                temp = base.copy()
                temp['Price_Offered'] = price
                temp['Free_Support_Offered'] = support
                temp['Demo_Type'] = demo_type

                temp = pd.DataFrame([temp])
                temp = pd.get_dummies(temp, columns=['Industry', 'Product_Type', 'Demo_Type'])

                # اطمینان از وجود همه ستون‌ها
                for col in needed_columns:
                    if col not in temp.columns:
                        temp[col] = 0
                temp = temp[needed_columns]

                prob = model.predict_proba(temp)[0][1]

                # هزینه‌ها
                setup = 50 if demo_type == 'Full' else 10
                supp_cost = support_cost(base['Company_Size']) if support == 1 else 0
                install = 200
                cost = setup + supp_cost + install

                profit = prob * price - cost

                if profit > best_profit:
                    best_profit = profit
                    best_offer_result = {
                        'make_offer': True,
                        'Demo_Type': demo_type,
                        'Free_Support_Offered': support,
                        'Price_Offered': price,
                        'Expected_Profit': profit,
                        'Predicted_Probability': prob
                    }

    return best_offer_result

# فایل داده‌ی مشتریان جدید
test_df = pd.read_excel("PotentialCust.xlsx")  # فقط ویژگی‌های مشتریان، بدون Demo یا Price یا Purchase
results = []

for _, row in test_df.iterrows():
    customer_id = row['Customer_ID']
    row_data = row.to_dict()
    decision = best_offer(row_data)
    decision['Customer_ID'] = customer_id
    results.append(decision)

# خروجی نهایی
results_df = pd.DataFrame(results)
results_df.to_csv("final_recommendations.csv", index=False)
